# Ciclystic Case Study

### **author:** João Carvalho

**date:** "04-07-2025"

## INTRODUCTION

In the capstone project of the Google Data Analytics Professional Certificate course on Coursera, a case study is presented for performing data analyst work for the fictional company Cyclistic.

<center>
  <img src="https://repository-images.githubusercontent.com/411734844/29248878-31fb-44a2-9c90-44986b6779c1" width="500">
</center>

- Scenario:
Cyclistic offers bike rental services with over 1,300 standalone bike rental stations located throughout the city of Chicago, with two types of subscription options: casual, which allows hourly and full-day rentals, and membership, which subscribers pay an annual fee and have free access to the service.

The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. Therefore, your team wants to understand how casual riders and annual members use Cyclistic bikes differently.

Through data analysis, I identified that casuals and members are part of different customer segments, with different habits and motivations for using the company’s service.

Thus, the insights found were presented through charts and 3 main recommendations were presented to the company's stakeholders: start directing different marketing strategies to users who use it as a means of transportation in their daily lives and those who use it for leisure and offer benefits for certain use of the service, such as future discounts for casuals who make longer rentals and accumulation of points in cold months to use in warm months.

![](https://i.imgur.com/XK7rt73.jpeg)

## ASK

Executives and marketing believe that converting casual subscribers to members will increase the company’s profitability. As a data analyst, I will analyze historical ride data from the previous 12 months to determine how casual subscribers and members are similar and different in their usage of the service. Through this analysis, I will find patterns and insights that will inform stakeholder decisions, marketing campaigns, and increase the company’s profitability.

## PREPARE

Data Location: The data used was provided through the Public Divvy bike-share dataset, downloaded from [Divvy’s official website](https://divvybikes.com/data-license-agreement). During the analysis, data related to [Chicago’s community](https://data.cityofchicago.org/Facilities-Geographic-Boundaries/Boundaries-Community-Areas-Map/cauq-8yn6) areas was also used.

Data Organization: Divvy’s files are categorized by month and in CSV format, containing details about the rides: ride ID, bike type, rental and return station names, station ID, rental and return latitude and longitude, and subscription type.

Data Credibility & Bias: The selected files were related to the period from 06, 2024 to 05, 2025 (202406-divvy_trip_data.csv to 202504-divvy_trip_data.csv), totaling 12 months, to have a more complete view of rental patterns over a one-year period for each type of service subscription.

Because the data is provided by a real bike-sharing company, it is considered reliable. To maintain the anonymity of the service's customers, personal data such as customer information, age, gender, financial transactions and others are not available, which makes it impossible to perform more in-depth analyses of the users.

The number of “members” subscriptions is practically double that of “casuals”, so the data was always analyzed separately. Some other inconsistencies were found, such as null fields, incorrectly filled out data, and values ​​that only appear in a specific month. This data was discarded and will be addressed in more depth throughout the case study.

## PROCESS

The tool chosen to clean and analyze the data was SQL in Bigquery. The results for each month were transferred to a spreadsheet to be analyzed separately, grouped into the total for the year and transformed into charts.

The first step was to import all the CSV spreadsheets into Bigquery.

For this markdown, we first need to create the environment and connect bigquery with jupter notebook.


In [ ]:
!pip install google-cloud-bigquery --upgrade

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Autenticaded')

Autenticaded


In [ ]:
project_id = "linear-skill-459817-k0"

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Create a BigQuery client
client = bigquery.Client(project=project_id)

**Data Cleaning:** In the data cleaning process, the data was checked for unique duplicates, cells with incorrect data input and missing data.

All the following steps were performed on all 12 data sets for each month.

### 1. Duplicate data:

- Check if the ride_ids were unique.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
      COUNT(*) AS total_rides,
      COUNT(DISTINCT ride_id) AS distinct_ride_id
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,total_rides,distinct_ride_id
0,502456,502456


The number of distinct ride_ids (distinct_ride_id) is equal to the total_rides, meaning there are no duplicates.

### 2. Incorrect Data:

- Checking the rideable_type and member_casual categories shows the values ​​contained in the project scope. rideable_type: “classic_bike” and “electric_bike” and member_casual: “member” and “casual”.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
      COUNT(DISTINCT rideable_type) AS distinct_types_bike,
      COUNT(DISTINCT member_casual) AS distinct_types_subscription
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_09_2024`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,distinct_types_bike,distinct_types_subscription
0,3,2


Only month 09 - 2024 presented a third type of bike “electric_scooter”. Another query was made to find out the quantity of this type of bike in the dataset.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT rideable_type,
      COUNT(*) AS total_per_rideable_type,
      -- calculate percentage in relation to sum categories
      ROUND(100 * COUNT(*) / SUM(COUNT(*)) OVER(), 2) AS percentual_type
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_09_2024`
GROUP BY rideable_type

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,rideable_type,total_per_rideable_type,percentual_type
0,classic_bike,314562,38.30
1,electric_bike,362377,44.12
2,electric_scooter,144337,17.57


All electric_scooter data was disregarded, about 17.57% of the data.

- Start time is equal to or greater than the end time of the rides.


In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
   COUNT(*) AS qnt_invalid_rides
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_11_2024`
WHERE started_at >= ended_at

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,qnt_invalid_rides
0,43


Only in month 11 - 2024 were rides found with finishing times lower than the starting time, approximately 0.012% of the total rides in the period. All of these rides were discarded.

- Number of rides that last less than 1 minute.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT member_casual,
      	COUNT(*) AS duration_minutes
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) = 0
GROUP BY member_casual

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,member_casual,duration_minutes
0,casual,6608
1,member,5754


The filter "WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) = 0” was added, creating the duration of the rides with the difference between the end time and the start time. Rides with less than 1 minute may represent internal tests, system errors and service cancellations. As they represent a small percentage (casual: 3.75% and member: 1.83%), these rides were discarded.

- Number of rides lasting more than 24 hours.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT MAX(TIMESTAMP_DIFF(ended_at, started_at, MINUTE)) AS max_minute
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_03_2025`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,max_minute
0,1559


First, a query was made to find the maximum times. According to the briefing, rentals have a maximum of 24 hours, equivalent to 1499 minutes. Since a number greater than 1499 was found, a new query was made to count the number of rides with the incorrect time.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT COUNT(*) AS qnt_rides_over_24
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_03_2025`
WHERE TIMESTAMP_DIFF(ended_at, started_at, HOUR) > 24

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,qnt_rides_over_24
0,9


In table 03 - 2025, there are 9 rides with a time of 25 hours (approximately 0.003% of the total). Some other months in 2024 have a maximum time of 1500 minutes, also with very low percentages. All these rows were discarded.

- Number of rides with coordinates (start_lat, start_lng, end_lat and end_lng) outside the city of Chicago.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT COUNT(*) AS total_out_chicago
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025` AS t1
-- join with community areas of Chicago dataset
LEFT JOIN `linear-skill-459817-k0.projeto_cyclistic.community_areas` AS t2
  ON ST_CONTAINS(ST_GEOGFROMTEXT(t2.the_geom), ST_GEOGPOINT(t1.start_lng, t1.start_lat))
WHERE t1.start_lat IS NOT NULL
  AND t1.start_lng IS NOT NULL
  AND t2.COMMUNITY IS NULL

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,total_out_chicago
0,8286


To perform this join, a table with coordinates for Chicago's community areas, provided by the Chicago Data Portal, was used. When analyzing geographic data, rides with locations outside the Chicago area were discarded.

### 3. Missing data:

- Percentage amount of missing values

All datasets have missing values ​​for start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat, end_lng. Queries were made to calculate the quantity and percentage of these values.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
    COUNT(*) AS total_rows,
    SUM(CASE
          WHEN start_station_name IS NULL THEN 1 ELSE 0 END) AS total_start_null,
          ROUND(100.0 * SUM(CASE WHEN start_station_name IS NULL THEN 1 ELSE 0 END) / COUNT(*), 2
    ) AS percentual_nulls
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,total_rows,total_start_null,percentual_nulls
0,502456,103360,20.57


The percentage of missing start_station_name fields ranges from 16.77% to 20.57%. Since they do not necessarily represent an error, when analyzing data directly related to these fields, rows with missing fields were discarded.


## ANALYSE

### 1. Number of rides per month by rental type

First, a query was made to count the valid runs (applying the cleaning filters to the inconsistencies found in the cleaning stage) for each type of signature.


In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
    member_casual,
    COUNT(*) AS qnt_rides_member_casual
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
-- filters for cleaning inconsistent data
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
      AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
      AND rideable_type <> "electric_scooter"
GROUP BY member_casual

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,member_casual,qnt_rides_member_casual
0,casual,176029
1,member,314065


Adding up the results of the 12 months, we arrive at the following numbers:
<center>
  <img src="https://i.imgur.com/Ya0tImQ.png">
</center>

The total number of rides per member is almost double that of casuals. Even if a report on the total number of rides were requested, it would be interesting to separate the two types of subscriptions to have a clearer view of the behaviors.

Then, a chart was created with the numbers from the last twelve months:
<center>
  <img src="https://i.imgur.com/PRVF8O0.png">
</center>

Looking at the chart, in the last months of the year, the number of rides drops sharply, with the lowest number in January, the coldest month in Chicago, with average temperatures between -6°C and 0°C. After February, with the temperature rising, the total number increases again.

Both casuals and members maintain the same pattern throughout the twelve-month period. However, members have a higher number even in low months. The month with the highest number of casual rides is July, with 310,473, and the lowest is January with 23,480, which represents a drop of 13.22x. The month with the highest number of member rides is August, with 430,390, and the lowest is January with 112,361, which represents a drop of 3.8x.

### 2. Average usage time

The following query converts the output and final data to find out the duration of the rides. From this data, the total and average ride time by subscription type is calculated.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
-- temporary table with the ride duration and filters to only count valid rides
WITH ride_duration_minutes as (
      SELECT
            started_at,
            ended_at,
            TIMESTAMP_DIFF(ended_at, started_at, MINUTE) AS duration_minutes,
            member_casual
      FROM `linear-skill-459817-k0.projeto_cyclistic.rides_03_2025`
      WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
            AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
            AND rideable_type <> "electric_scooter"
)
-- finding total rides, duration and average per ride type
SELECT
  member_casual,
  COUNT(*) AS total_per_member,
  SUM(duration_minutes) AS total_duration_ride,
  ROUND(AVG(duration_minutes), 2) AS avg_per_member
FROM ride_duration_minutes
GROUP BY member_casual

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,member_casual,total_per_member,total_duration_ride,avg_per_member
0,casual,83062,1734196,20.88
1,member,208518,2286227,10.96


Adding up the results for the entire year and putting them into charts:
<center>
  <img src="https://i.imgur.com/6NCpCLT.png" >
</center>

From this analysis, we can arrive at some insights:
- Even though the number of member rides is almost double that of casuals, the total number of minutes spent by casuals exceeds that of members.
- The average time spent by casuals is almost double that of members, inverting the first chart.
- Members take shorter rides and casuals take longer rides.


### 3. Ride duration

To get a broader idea of ​​the duration of rides, a query is made to discover the number and percentage of rides that last less and more than 40 minutes by subscription type.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
-- temporary table with the ride duration and filters to only count valid rides
WITH duration_rides AS (
  SELECT
    member_casual,
    TIMESTAMP_DIFF(ended_at, started_at, MINUTE) AS duration
  FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
  WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
  AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
  AND rideable_type <> "electric_scooter"
)
-- total and percentage of rides shorter and longer than 40 minutes per member type
SELECT
    member_casual,
    COUNTIF(duration < 40) AS less_40_min,
    ROUND(
        COUNTIF(duration < 40) * 100.0 / COUNT(*),
        2
    ) AS percent_less_40,
    COUNTIF(duration >= 40) AS longer_40_min,
    ROUND(
        COUNTIF(duration >= 40) * 100.0 / COUNT(*),
        2
    ) AS percent_longer_40
FROM duration_rides
GROUP BY member_casual

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,member_casual,less_40_min,percent_less_40,longer_40_min,percent_longer_40
0,casual,156421,88.86,19608,11.14
1,member,307349,97.86,6716,2.14


According to the result of the query above, related to the month 05 - 2025, we identified that a little over 2% of members' rides last more than 40 minutes, while for casuals this value exceeds 11%, indicating that members take faster rides, while casuals are more likely to enjoy the day with the service.

Comparing with the coldest month, the same query related to January:

In [ ]:
%%bigquery df --project linear-skill-459817-k0
-- temporary table with the ride duration and filters to only count valid rides
WITH duration_rides AS (
  SELECT
    member_casual,
    TIMESTAMP_DIFF(ended_at, started_at, MINUTE) AS duration
  FROM `linear-skill-459817-k0.projeto_cyclistic.rides_01_2025`
  WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
  AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
  AND rideable_type <> "electric_scooter"
)
-- total and percentage of rides shorter and longer than 40 minutes per member type
SELECT
    member_casual,
    COUNTIF(duration < 40) AS less_40_min,
    ROUND(
        COUNTIF(duration < 40) * 100.0 / COUNT(*),
        2
    ) AS percent_less_40,
    COUNTIF(duration >= 40) AS longer_40_min,
    ROUND(
        COUNTIF(duration >= 40) * 100.0 / COUNT(*),
        2
    ) AS percent_longer_40
FROM duration_rides
GROUP BY member_casual

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,member_casual,less_40_min,percent_less_40,longer_40_min,percent_longer_40
0,casual,22748,96.87,734,3.13
1,member,110721,98.54,1640,1.46


The results show a considerable drop for casual subscribers, with only 3%. On the other hand, the percentage of members did not drop as much, falling from 2.14% to 1.46%, and it seems that the weather does not generate many results in the distances traveled.

The chart below shows the proportion with the result added to the 12 months:
<center>
  <img src="https://i.imgur.com/l9GNeEd.png" >
</center>

These percentages can vary directly depending on the subscription options, since casuals pay per hour, favoring longer rides, while members can take more than one ride per day without changing the amount spent.

### 4. Total by days of the week

To analyze whether the behavior of the two types of subscription varies depending on the week, the rides were grouped by days of the week.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
-- extracting day of week from start date time
    EXTRACT(DAYOFWEEK FROM started_at) AS day_number,
-- converting day of week to word from start date time
    FORMAT_TIMESTAMP('%A', started_at) AS day_name,
    COUNTIF(member_casual = 'casual') AS total_casual_rides,
    COUNTIF(member_casual = 'member') AS total_member_rides,
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
    AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
    AND rideable_type <> "electric_scooter"
GROUP BY day_name, day_number
ORDER BY day_number

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,day_number,day_name,total_casual_rides,total_member_rides
0,1,Sunday,24554,28583
1,2,Monday,19451,38097
2,3,Tuesday,17796,44707
3,4,Wednesday,16670,43951
4,5,Thursday,24781,58029
5,6,Friday,32516,55173
6,7,Saturday,40261,45525


Chart relating to the 12-month period added together by subscription type:
<center>
  <img src="https://i.imgur.com/ssoqBpi.png" >
</center>

When we look at the chart, with opposing trends between the two groups, we can identify some insights:

- **The peak of casual riders is on the weekend,** with the three days with the highest volume being Saturday, Sunday and Friday, which may indicate that **the service is used more for leisure and on days off.**

- **The opposite effect is seen among members,** with the days with the most rides in the middle of the week, while Saturday and Sunday are days with the lowest number, which may indicate that they use the bike rental **service as a means of transportation in their daily lives, for study or work.**

### 5. Rides by time of day

To investigate further patterns, rides have now been grouped by time of day, taking into account the time at which bikes are picked up from the start stations.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
-- extracting hour from start date time
    EXTRACT(HOUR FROM started_at) AS hour_of_day,
    COUNTIF(member_casual = 'casual') AS total_casual_rides,
    COUNTIF(member_casual = 'member') AS total_member_rides
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
  AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
  AND rideable_type <> "electric_scooter"
GROUP BY hour_of_day
ORDER BY hour_of_day

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,hour_of_day,total_casual_rides,total_member_rides
0,0,3245,2830
1,1,1993,1718
2,2,1214,914
3,3,641,635
4,4,482,818
5,5,975,2949
6,6,2213,8551
7,7,4240,16752
8,8,5766,21387
9,9,6149,14327


<center>
  <img src="https://i.imgur.com/cSk3rSw.png" >
</center>

According to the chart, with the total number of rides from the 12 months of data, we can see different patterns between casuals and members. While casuals show an increasing number of pickups throughout the day and into the evening, peaking at 5 pm, the total number of members also increases throughout the day, but with two peaks, one in the morning from 7 am to 9 am and another from 4 pm to 6 pm. This supports the theory that they use Cyclistic's service as a means of transportation to and from work.

### 6. Ride times by days of week hours

To get a more in-depth result on this theory, the ride times have now been separated by day of the week.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
       EXTRACT(DAYOFWEEK FROM started_at) AS day_of_week,
       FORMAT_TIMESTAMP('%A', started_at) AS day_name,
       EXTRACT(HOUR FROM started_at) AS hour_of_day,
       COUNTIF(member_casual = 'casual') AS total_casual_rides,
       COUNTIF(member_casual = 'member') AS total_member_rides
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
  AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
  AND rideable_type <> "electric_scooter"
GROUP BY day_of_week, day_name, hour_of_day
ORDER BY day_of_week, hour_of_day

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,day_of_week,day_name,hour_of_day,total_casual_rides,total_member_rides
0,1,Sunday,0,804,660
1,1,Sunday,1,473,380
2,1,Sunday,2,277,221
3,1,Sunday,3,134,136
4,1,Sunday,4,86,102
...,...,...,...,...,...
163,7,Saturday,19,2031,2454
164,7,Saturday,20,1292,1633
165,7,Saturday,21,1141,1354
166,7,Saturday,22,1086,1157


<center>
  <img src="https://i.imgur.com/4w81GLR.png" >
</center>

When we analyze the charts, we can see how they support the theory that members use bicycles to go to work during the week, since the charts for weekdays differ from the casual ones, with two peaks, while on weekends they have similar shapes to the casual ones, probably with more leisure rides.

### 7. Quantity of rides per bike type

The next query calculates the number of rides by bike type and their percentage, to identify if there is any difference between subscription types.


In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
    member_casual,
    rideable_type,
    COUNT(*) AS total_member_type_rides,
-- count of percentage of rides by bike type relative to total member ride types
    ROUND(
        COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY member_casual),
        2
    ) AS percent_per_type
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
WHERE TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
    AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
    AND rideable_type <> "electric_scooter"
GROUP BY member_casual, rideable_type
ORDER BY member_casual

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,member_casual,rideable_type,total_member_type_rides,percent_per_type
0,casual,classic_bike,67335,38.25
1,casual,electric_bike,108694,61.75
2,member,classic_bike,118761,37.81
3,member,electric_bike,195304,62.19


<center>
  <img src="https://i.imgur.com/byn6dKw.png" >
</center>

Analyzing the results of the 12 months, the proportion between classic and electric bikes varies according to the months, but the proportion between casual and members always follows the same pattern with similar percentages, it does not seem to matter more for one group than for another.

### 8. Stations with the most rides

In order to begin the geographical exploration of each type of subscription, a query was carried out to discover the top 10 most accessed start stations.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
-- top 10 start stations filtering by member type
SELECT
    start_station_name,
    COUNT(*) AS total_start
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
WHERE member_casual = 'casual'
      AND start_station_name IS NOT NULL
      AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
      AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
      AND rideable_type <> "electric_scooter"
GROUP BY start_station_name
ORDER BY total_start DESC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,start_station_name,total_start
0,Streeter Dr & Grand Ave,4001
1,DuSable Lake Shore Dr & Monroe St,3196
2,Michigan Ave & Oak St,1832
3,Millennium Park,1831
4,Shedd Aquarium,1824
5,Dusable Harbor,1810
6,DuSable Lake Shore Dr & North Blvd,1637
7,Theater on the Lake,1225
8,Field Museum,1098
9,Indiana Ave & Roosevelt Rd,1027


Grouping the results of a one-year period:
<center>
  <img src="https://i.imgur.com/Zxo4EUE.png" >
</center>

The top stations in each group differ greatly:

- The top 10 casuals are located **near tourist attractions and the coast, i.e. leisure and tourism areas.**

- While the top 10 members, even in the downtown area of ​​Chicago, **indicate more internal streets, near train terminals, shopping centers, colleges and residential buildings.**

Analyzing the set of rides as a whole, we can see that the percentage of the most accessed stations is very small compared to the total set. Even so, casuals access their main departure station almost 3x more than members.

| TYPE   | TOTAL RIDES | TOTAL TOP STATION | PERCENTAGE |
|:-------|------------:|------------------:|-----------:|
| Casual |     1917536 |             46368 |      2,42% |
| Member |     3450470 |             30346 |      0,87% |

### 9. Community Areas with the most rides

To get a broader geographic picture, the starting stations were grouped by community areas of Chicago, using the geographic boundaries of latitude and longitude provided by the Chicago Data Portal, to compare user patterns.

In [ ]:
%%bigquery df --project linear-skill-459817-k0
-- top 10 community areas filtering by member type
SELECT
  t2.COMMUNITY AS start_community,
  count(*) as qnt_rides_per_community
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025` AS t1
-- join with community areas of Chicago dataset
JOIN `linear-skill-459817-k0.projeto_cyclistic.community_areas` AS t2
ON ST_CONTAINS(ST_GEOGFROMTEXT(t2.the_geom), ST_GEOGPOINT(t1.start_lng, t1.start_lat))
WHERE member_casual = 'casual'
  AND t1.start_lat IS NOT NULL
  AND t1.start_lng IS NOT NULL
  AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
  AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
  AND rideable_type <> "electric_scooter"
GROUP BY t2.COMMUNITY
ORDER BY qnt_rides_per_community DESC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,start_community,qnt_rides_per_community
0,NEAR NORTH SIDE,34544
1,LOOP,25117
2,LINCOLN PARK,20863
3,LAKE VIEW,17953
4,NEAR WEST SIDE,14628
5,WEST TOWN,10752
6,NEAR SOUTH SIDE,7781
7,LOGAN SQUARE,6479
8,UPTOWN,5717
9,HYDE PARK,5328


Chart for one-year period by subscription type:
<center>
  <img src="https://i.imgur.com/Gb3EyCz.png" >
</center>

Comparison chart between the two types of subscription:
<center>
  <img src="https://i.imgur.com/Xu39CgL.png" >
</center>

When we look at the bigger picture, the regions used by casuals and members are quite similar, appearing in roughly the same order, indicating a similar pattern.
<center>
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/Chicago_community_areas_map.svg/800px-Chicago_community_areas_map.svg.png" width="600">
</center>

We can also see that most of the operations by the two subscription groups are concentrated in the central and coastal areas of the city and surrounding areas. These regions have the highest population densities, as well as public transport terminals, many leisure areas and tourist attractions.

### 10. Routes with the most rides

To better understand the motivation behind rides by each subscription group, a new query was created to discover the top 5 most popular routes.


In [ ]:
%%bigquery df --project linear-skill-459817-k0
SELECT
      start_station_name,
      end_station_name,
      COUNT(*) AS qnt_of_route
FROM `linear-skill-459817-k0.projeto_cyclistic.rides_05_2025`
WHERE member_casual = 'casual'
      AND start_station_name IS NOT NULL
      AND end_station_name IS NOT NULL
      AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) > 0
      AND TIMESTAMP_DIFF(ended_at, started_at, MINUTE) < 1500
      AND rideable_type <> "electric_scooter"
GROUP BY start_station_name, end_station_name
ORDER BY qnt_of_route DESC
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df

,start_station_name,end_station_name,qnt_of_route
0,DuSable Lake Shore Dr & Monroe St,DuSable Lake Shore Dr & Monroe St,644
1,Streeter Dr & Grand Ave,Streeter Dr & Grand Ave,635
2,DuSable Lake Shore Dr & Monroe St,Streeter Dr & Grand Ave,518
3,Michigan Ave & Oak St,Michigan Ave & Oak St,312
4,Dusable Harbor,Dusable Harbor,305


<center>
  <img src="https://i.imgur.com/0vQMBo4.png" >
</center>

Comparing the most popular routes, we can draw some insights:

- **Casuals tend to rent and return their bikes at the same station,** which indicates that they use the service more for leisure and physical activity than for getting around, since they return to the starting point. Furthermore, all of these locations are **close to tourist attractions and on the coast.**

- Contrariwise, **members have all the start stations different from the end stations,** indicating that they use the service to get around. The most accessed points are still located in the most **central area of ​​the city, close to universities.**

- This data contributes to the indication that casuals rent for leisure purposes, while members use it as a means of transportation around the city, not only to work, but also to study.

## ACT

Analyzing the files for the last 12 months, similarities and differences in the patterns between the two types of signatures were found.

- Similarities:

The type of bike and community areas with the most rides are practically the same in both groups. Although they vary according to the months, the type of bike remains proportional to the type of subscription, with a constant preference for electric bikes. The community areas appear in practically the same order in both groups, concentrated in places with the largest demographic number, in the city's central neighborhoods, locations of many business centers, colleges, public transport terminals and tourist and leisure spots.

- Differences:

The differences found in both subscription types indicate that members and casuals have different motivations for using the service, while casuals use it for leisure and members use it as a means of transportation.

Even though the total number of rides by members is almost double that of casuals, the total time spent using the service by casuals exceeds that of members, with an average ride time of almost double (22 minutes vs. 11 minutes). These numbers are due to the fact that members make faster trips from one place to another in the city, while casuals take advantage of the rental for longer periods.

The main factor confirming the theory of different motivations by subscription group is the analysis of behavior patterns throughout the week. Casuals use the service more on weekends, while members take more rides during the week. Observing the ride times, members have two peaks of rides throughout the day on weekdays, during the time they start and finish classes and work, while casuals have an increasing demand throughout the day until the early evening. On weekends, the times of both groups are the same, that is, on these days they use the service for similar purposes.

In the cold months, even with a large drop in the number of total rides, the number of rides for members drops 3.8x compared to the month with the highest number, while the number of rides for casuals drops 13.2x. The difference in these numbers indicates that members are more loyal to the service than casuals, since they use the service as a means of transportation for their daily tasks, regardless of the weather.

The last point found that indicates different purposes is the most common routes taken by group type. Casuals rent at a station and return to the same station (mostly on the coast and at tourist spots), that is, they rent without any commitment to travel, opting to go for a walk or do physical activity, then returning to the starting point. Conversely, members rent at stations further into the central neighborhoods and return to different stations, generally on routes of 4 - 7 minutes, since they are using the service as a means of transportation to go to work or school.

- Other insights found:

In general, the data related to the rides were satisfactory for carrying out the work, but more complete data, with personal and geographic information about the customers (user ID, gender, age, profession, place of residence, whether they are from the city or not, etc.) would help to better support the insights found with the observed patterns of the rides in relation to two market segments.

For future and more in-depth analyses, more data would be necessary. Information entered by users when subscribing could be incorporated. Implementing satisfaction questionnaires at the end of the rides could also increase the company's datasets.

Therefore, after analyzing the similarities and differences between the two types of subscription, 3 main recommendations were drawn up for the company's stakeholders.


### Top 3 Recommendations:

- Start directing different marketing strategies to the two customer segments, those who use it for daily commuting and those who use it for leisure.

Member: Campaigns offering alternatives for those who don't want to take public transportation or sit in traffic and always have the option of cycling. Offer partnerships with universities and local companies to offer corporate plans as a benefit for students and employees.

Casual: Local campaigns at tourist attractions and leisure areas on weekends. Partnerships with tourism services and groups. Seasonal promotions before holidays and vacation periods.

Monitor the stations most used by each type of user through data to implement different physical campaigns.

- Offer rewards through benefits, such as discounts on future rides for casual users who prefer to use the service for longer rides, so that they will return to use the service on another day.

- Invest in opportunities for the future during cold months, as service usage numbers fall, such as progressive discounts, accumulation of points to be used in the following months and increasing the number of rides more quickly in the warm months.
